In [147]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix,ConfusionMatrixDisplay

In [148]:

train_dir='/Users/hiten/Documents/Maincode/Dataset/train'
valid_dir='/Users/hiten/Documents/Maincode/Dataset/val'
test_dir='/Users/hiten/Documents/Maincode/Dataset/test'

Size=224

In [149]:
import tensorflow as tf
train_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    rescale=1/255.0,
).flow_from_directory(train_dir,batch_size=16,target_size=(Size,Size),subset="training",shuffle=True)

valid_generator=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255.0,
).flow_from_directory(valid_dir,batch_size=16,target_size=(Size,Size),shuffle=False)

Found 42230 images belonging to 38 classes.
Found 9042 images belonging to 38 classes.


In [175]:
new_model = tf.keras.models.load_model(f'Inceptionresnetv2_2022-09-08')

In [176]:
# Evaluate the restored model
loss, acc = new_model.evaluate(valid_generator)

2022-09-09 00:54:31.364131: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


566/566 [==============================] - 167s 284ms/step - loss: 0.2653 - accuracy: 0.9102


In [177]:
predictions = np.argmax(new_model.predict(valid_generator),axis=1)

2022-09-09 00:57:19.161333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


566/566 [==============================] - 168s 287ms/step


In [178]:
predictions = predictions.astype("int32")

In [179]:
valid_generator.classes

array([ 0,  0,  0, ..., 37, 37, 37], dtype=int32)

In [180]:
def counts_from_confusion(confusion):
    """
    Obtain TP, FN FP, and TN for each class in the confusion matrix
    """

    counts_list = []

    # Iterate through classes and store the counts
    for i in range(confusion.shape[0]):
        tp = confusion[i, i]

        fn_mask = np.zeros(confusion.shape)
        fn_mask[i, :] = 1
        fn_mask[i, i] = 0
        fn = np.sum(np.multiply(confusion, fn_mask))

        fp_mask = np.zeros(confusion.shape)
        fp_mask[:, i] = 1
        fp_mask[i, i] = 0
        fp = np.sum(np.multiply(confusion, fp_mask))

        tn_mask = 1 - (fn_mask + fp_mask)
        tn_mask[i, i] = 0
        tn = np.sum(np.multiply(confusion, tn_mask))

        counts_list.append({'Class': i,
                            'TP': tp,
                            'FN': fn,
                            'FP': fp,
                            'TN': tn})

    return counts_list

In [181]:
from sklearn.metrics import top_k_accuracy_score,cohen_kappa_score,matthews_corrcoef,recall_score

cm = confusion_matrix(valid_generator.classes,predictions)


counting_cm_list=counts_from_confusion(cm)
counting_cm_list

[{'Class': 0, 'TP': 126, 'FN': 24.0, 'FP': 14.0, 'TN': 8878.0},
 {'Class': 1, 'TP': 140, 'FN': 10.0, 'FP': 8.0, 'TN': 8884.0},
 {'Class': 2, 'TP': 147, 'FN': 3.0, 'FP': 5.0, 'TN': 8887.0},
 {'Class': 3, 'TP': 220, 'FN': 26.0, 'FP': 26.0, 'TN': 8770.0},
 {'Class': 4, 'TP': 220, 'FN': 5.0, 'FP': 11.0, 'TN': 8806.0},
 {'Class': 5, 'TP': 146, 'FN': 11.0, 'FP': 9.0, 'TN': 8876.0},
 {'Class': 6, 'TP': 143, 'FN': 7.0, 'FP': 13.0, 'TN': 8879.0},
 {'Class': 7, 'TP': 123, 'FN': 27.0, 'FP': 11.0, 'TN': 8881.0},
 {'Class': 8, 'TP': 174, 'FN': 4.0, 'FP': 7.0, 'TN': 8857.0},
 {'Class': 9, 'TP': 132, 'FN': 18.0, 'FP': 27.0, 'TN': 8865.0},
 {'Class': 10, 'TP': 174, 'FN': 0.0, 'FP': 7.0, 'TN': 8861.0},
 {'Class': 11, 'TP': 156, 'FN': 21.0, 'FP': 24.0, 'TN': 8841.0},
 {'Class': 12, 'TP': 187, 'FN': 20.0, 'FP': 18.0, 'TN': 8817.0},
 {'Class': 13, 'TP': 159, 'FN': 2.0, 'FP': 6.0, 'TN': 8875.0},
 {'Class': 14, 'TP': 146, 'FN': 4.0, 'FP': 4.0, 'TN': 8888.0},
 {'Class': 15, 'TP': 817, 'FN': 9.0, 'FP': 12.0, 

In [182]:
import pandas as pd
from tqdm.notebook import tqdm
Sensitivity = []
Specificity = []
class_accuracy =[]
class_label = []
Recall = []
Precision = []
F1_score = []

for i in tqdm(range(38)):
    tp = counting_cm_list[i]['TP']
    tn = counting_cm_list[i]['TN']
    fp = counting_cm_list[i]['FP']
    fn = counting_cm_list[i]['FN']
    
    
    # sensitivity
    senti = (tp / (tp + fn))*100
    senti = "{:.2f}".format(senti)
    #specificity
    speci = (tn / (tn + fp))*100
    speci = "{:.2f}".format(speci)
    
    #Precision
    precision_new = "{:.2f}".format((tp/(tp+fp))*100)
    #recall
    recall_new = "{:.2f}".format((tp/(tp+fn))*100)
    #F1-Score
    
    f1_score_new = 2*(float(precision_new)*float(recall_new))/(float(recall_new)+float(precision_new))
    f1_score_new = "{:.2f}".format(f1_score_new)
    
    # Overall accuracy
    ACC = ((tp + tn)/(tp+fp+fn+tn))*100
    ACC = "{:.2f}".format(ACC)
    class_label.append(i)
    class_accuracy.append(ACC)
    Sensitivity.append(senti)
    Specificity.append(speci)
    Precision.append(precision_new)
    Recall.append(recall_new)
    F1_score.append(f1_score_new)
    


  0%|          | 0/38 [00:00<?, ?it/s]

In [183]:

recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)


total_specificity = [float(i) for i in Specificity]
total_specificity = np.mean(total_specificity)
print("Specificity: ",total_specificity,"%")

total_Sensitivity = [float(i) for i in Sensitivity]
total_Sensitivity = np.mean(total_Sensitivity)
print("Sensitivity: ",total_Sensitivity,"%")

total_recall = np.mean(recall)
total_precision = np.mean(precision)
total_f1_score = 2*(total_precision*total_recall)/(total_precision+total_recall)

print("Recall: ", "{:.2f}".format(total_recall*100),"%")
print("Precision: ", "{:.2f}".format(total_precision*100),"%")
print("F1-Score: ", "{:.2f}".format(total_f1_score*100),"%")



Specificity:  99.7578947368421 %
Sensitivity:  89.98105263157895 %
Recall:  89.98 %
Precision:  89.46 %
F1-Score:  89.72 %


In [184]:
from sklearn.metrics import top_k_accuracy_score,cohen_kappa_score,matthews_corrcoef,recall_score
from sklearn.metrics import classification_report,cohen_kappa_score,matthews_corrcoef
print(f"Model Result InceptionV3")

print(classification_report(valid_generator.classes, predictions))

cohen_kappa_score = cohen_kappa_score(valid_generator.classes, predictions)
Metthews_correlation = matthews_corrcoef(valid_generator.classes, predictions)
# Top_K_accuracy = top_k_accuracy_score(valid_generator.classes, predictions, k=1)

print("Cohen Kappa Score: ", cohen_kappa_score)
print("Matthews correlation coefficient : ", Metthews_correlation)
# print("Top k Accuracy Score: ",Top_K_accuracy)

Model Result InceptionV3
              precision    recall  f1-score   support

           0       0.90      0.84      0.87       150
           1       0.95      0.93      0.94       150
           2       0.97      0.98      0.97       150
           3       0.89      0.89      0.89       246
           4       0.95      0.98      0.96       225
           5       0.94      0.93      0.94       157
           6       0.92      0.95      0.93       150
           7       0.92      0.82      0.87       150
           8       0.96      0.98      0.97       178
           9       0.83      0.88      0.85       150
          10       0.96      1.00      0.98       174
          11       0.87      0.88      0.87       177
          12       0.91      0.90      0.91       207
          13       0.96      0.99      0.98       161
          14       0.97      0.97      0.97       150
          15       0.99      0.99      0.99       826
          16       0.96      0.95      0.96       344
  

In [185]:
df = pd.DataFrame()
df["Class_Label"] = class_label
df["Specificity(%)"] = Specificity
df["Sensitivity(%)"] = Sensitivity
df["Precision(%)"] = Precision
df["Recall(%)"] = Recall
df["F1_score(%)"] = F1_score
df["Accuracy(%)"] = class_accuracy

df2 = pd.DataFrame({'Class_Label': ['Total scores'],
                    'Specificity(%)':[total_specificity],
                    'Sensitivity(%)':[total_Sensitivity],
                    'Precision(%)':[total_precision],
                    'Recall(%)':[total_recall],
                    'F1_score(%)':[total_f1_score],
                    'Accuracy(%)':[acc],
                    
                    })

df3 = pd.DataFrame({'Class_Label': ['Total scores'],
                    'Specificity(%)':[total_specificity],
                    'Sensitivity(%)':[total_Sensitivity],
                    'Precision(%)':[total_precision],
                    'Recall(%)':[total_recall],
                    'F1_score(%)':[total_f1_score],
                    'Accuracy(%)':[acc],
                    
                    })
df = pd.concat([df, df2], ignore_index = True, axis = 0)

In [186]:
df

,Class_Label,Specificity(%),Sensitivity(%),Precision(%),Recall(%),F1_score(%),Accuracy(%)
0,0,99.84,84.00,90.00,84.00,86.90,99.58
1,1,99.91,93.33,94.59,93.33,93.96,99.80
2,2,99.94,98.00,96.71,98.00,97.35,99.91
3,3,99.70,89.43,89.43,89.43,89.43,99.42
4,4,99.88,97.78,95.24,97.78,96.49,99.82
5,5,99.90,92.99,94.19,92.99,93.59,99.78
6,6,99.85,95.33,91.67,95.33,93.46,99.78
7,7,99.88,82.00,91.79,82.00,86.62,99.58
8,8,99.92,97.75,96.13,97.75,96.93,99.88
9,9,99.70,88.00,83.02,88.00,85.44,99.50


In [187]:
df.to_csv('CSV_files/InceptionResnetV2.csv')